In [ ]:
#!pip install pandas
#!pip install indian-names
#!pip install random-word
#!pip install psycopg2
#!pip install faker
#!pip install pyarrow
#!pip install sqlalchemy
#!pip install scikit-learn

In [ ]:
#imports
import pandas as pd
import indian_names
import random
import psycopg2
from random_word import RandomWords
from datetime import datetime, timedelta
import pyarrow as pa
import pyarrow.parquet as pq
import os
import json
from collections import Counter
from faker import Faker
from sqlalchemy import create_engine
fake = Faker()

In [ ]:
def generate_aadhar_number():
    return ' '.join(''.join(map(str,random.sample(range(10), 4))) for x in range(3))

In [ ]:
def generate_passport_number():
    alphabets = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
    letter = random.choices(alphabets)[0]
    number =letter + str(random.randint(1,9)) + str(random.randint(0,9))\
                + ''.join(map(str,random.sample(range(10),4))) + str(random.randint(1,9)) 
    return number

In [ ]:
def generate_phone_number():
    phone_number = ''
    
    #first number should be between 6-9
    phone_number += (str(random.randint(7,9)))
    
    for i in range(1, 10): 
        phone_number += (str(random.randint(0, 9))) 
        
    return phone_number

In [ ]:
def random_with_N_digits(n):
    range_start = 10**(n-1)
    range_end = (10**n)-1
    return random.randint(range_start, range_end)

In [ ]:
class Employee:
    def __init__(self,departmanet,designation,employeeID):
        self.gender = random.choice(["male","female"])
        self.first_name = indian_names.get_first_name(self.gender)
        self.last_name = indian_names.get_last_name()
        if(designation == 'pilot' or designation == 'co-pilot'):
            self.age = random.randint(25, 35)
        elif(designation == 'crew-member'):
            self.age = random.randint(22, 35)
        else:
            self.age = random.randint(22, 64)
        self.contact_number = generate_phone_number()
        self.email_id = self.first_name + self.last_name + '.' + employeeID + '@zathura.com'
        self.aadhar_number = generate_aadhar_number()
        self.department = departmanet
        self.designation = designation
        self.employee_id = employeeID

    def get(self):
        return {
            'employee_id' : self.employee_id,
            'first_name' : self.first_name,
            'last_name' : self.last_name,
            'age' : self.age,
            'gender' : self.gender,
            'email' : self.email_id,
            'contact_number' : self.contact_number,
            'department' : self.department,
            'designation' : self.designation,
            'aadhar' : self.aadhar_number
        }


In [ ]:
class Crew:
    def __init__(self):
        self.employees = []
        self.employee_id = 10001
        self.airplane_metadata = {
            'Garuda' : {'pilot' : 2 ,'co-pilot' : 2, 'crew-member' : random.randint(7,10)}, 
            'Vimana' : {'pilot' : 2 ,'co-pilot' : 2, 'crew-member' : random.randint(7,10)} ,
            'Vayu' : {'pilot' : 2 ,'co-pilot' : 2, 'crew-member' : random.randint(7,10)},
            'Ananta' : {'pilot' : 2 ,'co-pilot' : 2, 'crew-member' : random.randint(7,10)} ,
            'Agni': {'pilot' : 2 ,'co-pilot' : 2, 'crew-member' : random.randint(7,10)},
            'Astra' : {'pilot' : 2 ,'co-pilot' : 2, 'crew-member' : random.randint(7,10)},
            'Vajra' :{'pilot' : 2 ,'co-pilot' : 2, 'crew-member' : random.randint(7,10)} ,
            'Rudra' : {'pilot' : 2 ,'co-pilot' : 2, 'crew-member' : random.randint(7,10)},
            'Marut' : {'pilot' : 2 ,'co-pilot' : 2, 'crew-member' : random.randint(7,10)}
        }
        self.airplanes = ['Garuda', 'Vimana', 'Vayu', 'Ananta','Agni', 'Astra', 'Vajra','Rudra', 'Marut']

        self.departments = ['Finance','Traffic-control-tower','Infrastructure-Maintenance','HR','IT']
        self.roles = {
            'Finance' : { 'num' : 30 ,'designation' : ['Consultant','Senior Consultant','Specialist','Associate Director','Director'], 'weights' : [0.4,0.3,0.2,0.07,0.03]},
            'Traffic-control-tower' : { 'num' : 25,'designation' :['Junior-Connector','Ground-Controller','Senior-Connector','Chief'] , 'weights' : [0.5,0.3,0.2,0.1]},
            'Infrastructure-Maintenance' : { 'num' : 50, 'designation' :['IM-1','IM-2','IM-3','IM-4'], 'weights' : [0.4,0.3,0.2,0.01]},
            'HR': { 'num' : 20, 'designation' :['Recruitment','HR-assisstant','HR-Coordiantor','Chief people officer','HR Business Partner','HR Specialist'], 'weights' : [0.2,0.1,0.2,0.1,0.2,0.2]},
            'IT': { 'num' : 50, 'designation' : ['Developer','Tester','Engineers','Manager','Director']  , 'weights' : [0.3,0.1,0.4,0.1,0.1]},
        }

    def set_crew(self):
        for plane in self.airplanes:
            pilot = self.airplane_metadata[plane]['pilot']
            co_pilot = self.airplane_metadata[plane]['co-pilot']
            crew_members = self.airplane_metadata[plane]['crew-member']

            for _ in range(pilot):
                self.employees.append(Employee(f'flight-crew-{plane}','pilot',f'za-{self.employee_id}').get())
                self.employee_id = self.employee_id+ 1

            for _ in range(co_pilot):
                self.employees.append(Employee(f'flight-crew-{plane}','co-pilot',f'za-{self.employee_id}').get())
                self.employee_id = self.employee_id+ 1

            for _ in range(crew_members):
                self.employees.append(Employee(f'flight-crew-{plane}','crew-member',f'za-{self.employee_id}').get())
                self.employee_id = self.employee_id+ 1


    def set_employees(self):
        
        for department in self.departments:
            designation = self.roles[department]['designation']
            weights = self.roles[department]['weights']
            num = self.roles[department]['num']
            

            randomized_designation = random.choices(designation, weights = weights,k = num)

            print(f"designation - {designation} - {weights}")

            for i in range(num):
                self.employees.append(Employee(department,randomized_designation[i],f'za-{self.employee_id}').get())
                self.employee_id = self.employee_id+ 1

        
            

In [ ]:


choices = ['Consultant', 'Senior Consultant', 'Specialist', 'Associate Director', 'Director']
weights = [0.4, 0.3, 0.2, 0.07, 0.03]

# Get number of choices based on weights
num_choices = 10  # Change this to the desired number of choices

# Generate choices based on weights
result = random.choices(choices, weights=weights, k=num_choices)

print("Random choices based on weights:")
print(result)

In [ ]:
crew = Crew()
crew.set_crew()
crew.set_employees()
df = pd.DataFrame(crew.employees)

In [ ]:
import pandas as pd
import pyodbc


# Define your connection string
connection_string = "*****"

# Establish connection
try:
    conn = pyodbc.connect(connection_string)
    cursor = conn.cursor()

    # Convert DataFrame to list of tuples
    rows = [tuple(x) for x in df.values]

    # Construct the placeholder string
    placeholders = ', '.join(['?' for _ in range(len(df.columns))])

    # Construct the INSERT INTO statement with parameterized query
    insert_query = f"INSERT INTO Employees ({', '.join(df.columns)}) VALUES ({placeholders})"

    # Execute the INSERT statement with executemany
    cursor.executemany(insert_query, rows)

    # Commit the transaction
    conn.commit()

    print("DataFrame successfully inserted into SQL Server.")
except Exception as e:
    print("Error:", e)
finally:
    # Close connection
    if conn:
        conn.close()
